In [1]:
import torch
import torch.nn.functional as F
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn
from torch.utils.data import DataLoader
from model import Encoding_Network
from Triplet_Dataset import TripletData

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
device

device(type='cuda')

In [39]:
class Longer_Encoding_Network(nn.Module):
    
    def __init__(self):
        
        super(Longer_Encoding_Network, self).__init__()
        self.conv = nn.Sequential(nn.Conv2d(7, 32, kernel_size=2), nn.ReLU(), nn.Conv2d(32, 64, kernel_size=2), nn.BatchNorm2d(64), nn.ReLU(),
                                 nn.MaxPool2d(2,2))

        self.linear = nn.Linear(576, 784)
        
        
    def forward(self,x):
        x = self.conv(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.linear(x))
      
        return x
    
class ScoreNetL3(nn.Module):
    def __init__(self, Encoder_Path):
        super(ScoreNet, self).__init__()
        self.Enc = Longer_Encoding_Network()
        self.Enc.load_state_dict(torch.load(Encoder_Path))
        
        self.linear = nn.Sequential(nn.Linear(1568, 2048), nn.BatchNorm1d(2048), nn.ReLU(), nn.Linear(2048,512),
                                    nn.BatchNorm1d(512), nn.ReLU(), nn.Linear(512,1))
        
    def forward(self,Anchor, Sample):
        AnchorVec = self.Enc(Anchor)
        SampleVec = self.Enc(Sample)
        x = torch.cat((AnchorVec, SampleVec), 1)
        x = self.linear(x)
        return x

In [40]:
dset =  TripletData('training_neighbors.csv', True)
trainset, testset = dset.split(.95)

trainloader= torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=0 )
testloader= torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=0 )

Train Size: 4758327
Test Size: 250439


In [41]:
def train(model,criterion,optimizer,num_epochs,trainloader,testloader):
    
    for epoch in range(num_epochs):
        running_loss=0.0
        
        for i , (label, anchor, sample) in enumerate(trainloader):
            anchor = anchor.to(device)
            sample = sample.to(device)
            label = label.to(device)
            label = label.unsqueeze(1)
            label = label.to(torch.float32)
            
           
            #forward prop
            
            score = model(anchor,sample)
            
            loss = criterion(score, label)

            #backprop
            optimizer.zero_grad()
            loss.backward()

            #take step
            optimizer.step()
            losss= loss.item()
            
            running_loss+=losss
            
            
           
            if i %500==499:
                print('[%d,%5d] loss: %.3f' % (epoch+1 , i+1, running_loss/500))
                running_loss=0.0
            if i % 2000 == 1999: 
                print("Train accuracy: "+ str(check_accuracy(trainloader, model)*100))
                print("Test accuracy: "+ str(check_accuracy(testloader, model)*100))
            
            

In [42]:
def check_accuracy(loader, model, full=False):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for i , (label, anchor, sample) in enumerate(trainloader):
            if i > 200:
                model.train()
                return num_correct/num_samples
                
            anchor = anchor.to(device)
            sample = sample.to(device)
            label = label.to(device)
            label = label.unsqueeze(1)
            label = label.to(torch.float32)
            
           
            #forward prop
            
            score = model(anchor,sample)
            
            
            predictions = torch.sigmoid(score) > .5
            
            
            num_correct += (predictions == label).sum()
            
         
            
            
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples

In [43]:
model = ScoreNet('./previous_models/LVec2.pth').to(device)

In [ ]:
batch_size = 16
learning_rate = .5e-3
num_epochs = 10
criterion3 = nn.BCEWithLogitsLoss()
optimizer3=optim.Adam(model.parameters(),lr=learning_rate)
#optimizer3 = optim.SGD(my_model.parameters(),lr=learning_rate,momentum=.9)
num_epochs=15
train(model,criterion3,optimizer3,num_epochs,trainloader,testloader)

[1,  500] loss: 0.365
[1, 1000] loss: 0.358
[1, 1500] loss: 0.377
[1, 2000] loss: 0.371
Train accuracy: tensor(83.7376, device='cuda:0')
Test accuracy: tensor(84.2351, device='cuda:0')
[1, 2500] loss: 0.366
[1, 3000] loss: 0.371
[1, 3500] loss: 0.357
[1, 4000] loss: 0.368
Train accuracy: tensor(84.2662, device='cuda:0')
Test accuracy: tensor(83.7065, device='cuda:0')
[1, 4500] loss: 0.362
[1, 5000] loss: 0.357
[1, 5500] loss: 0.366
[1, 6000] loss: 0.371
Train accuracy: tensor(84.1729, device='cuda:0')
Test accuracy: tensor(84.1418, device='cuda:0')
[1, 6500] loss: 0.373
[1, 7000] loss: 0.373
[1, 7500] loss: 0.372
[1, 8000] loss: 0.369
Train accuracy: tensor(83.8619, device='cuda:0')
Test accuracy: tensor(82.4938, device='cuda:0')
[1, 8500] loss: 0.364
[1, 9000] loss: 0.367
[1, 9500] loss: 0.371
[1,10000] loss: 0.365
Train accuracy: tensor(84.1729, device='cuda:0')
Test accuracy: tensor(84.2040, device='cuda:0')
[1,10500] loss: 0.375
[1,11000] loss: 0.376
[1,11500] loss: 0.370
[1,12000]

[1,90000] loss: 0.364
Train accuracy: tensor(83.0846, device='cuda:0')
Test accuracy: tensor(83.5510, device='cuda:0')
[1,90500] loss: 0.372
[1,91000] loss: 0.367
[1,91500] loss: 0.364
[1,92000] loss: 0.359
Train accuracy: tensor(83.9863, device='cuda:0')
Test accuracy: tensor(83.1157, device='cuda:0')
[1,92500] loss: 0.360
[1,93000] loss: 0.363
[1,93500] loss: 0.371
[1,94000] loss: 0.370
Train accuracy: tensor(83.6443, device='cuda:0')
Test accuracy: tensor(83.7687, device='cuda:0')
[1,94500] loss: 0.361
[1,95000] loss: 0.364
[1,95500] loss: 0.365
[1,96000] loss: 0.369
Train accuracy: tensor(82.9913, device='cuda:0')
Test accuracy: tensor(83.6443, device='cuda:0')
[1,96500] loss: 0.364
[1,97000] loss: 0.367
[1,97500] loss: 0.366
[1,98000] loss: 0.360
Train accuracy: tensor(83.4577, device='cuda:0')
Test accuracy: tensor(82.8047, device='cuda:0')
[1,98500] loss: 0.360
[1,99000] loss: 0.372
[1,99500] loss: 0.369
[1,100000] loss: 0.362
Train accuracy: tensor(84.7326, device='cuda:0')
Tes

In [47]:
torch.save(model.state_dict(), './previous_models/ScoreNetL3-83.pth')

In [ ]:
'''./previous_models/LVec1.pth

./previous_models/ScoreNet1.pth
class ScoreNet(nn.Module):
    def __init__(self, Encoder_Path):
        super(ScoreNet, self).__init__()
        self.Enc = Encoding_Network()
        self.Enc.load_state_dict(torch.load(Encoder_Path))
        
        self.linear = nn.Sequential(nn.Linear(512, 1))
        
    def forward(self,Anchor, Sample):
        AnchorVec = self.Enc(Anchor)
        SampleVec = self.Enc(Sample)
        x = torch.cat((AnchorVec, SampleVec), 1)
        x = self.linear(x)
        return x
./previous_models/ScoreNet2.pth        
class ScoreNet(nn.Module):
    def __init__(self, Encoder_Path):
        super(ScoreNet, self).__init__()
        self.Enc = Encoding_Network()
        self.Enc.load_state_dict(torch.load(Encoder_Path))
        
        self.linear = nn.Sequential(nn.Linear(512, 128), nn.BatchNorm1d(128), nn.ReLU(), nn.Linear(128,1))
        
    def forward(self,Anchor, Sample):
        AnchorVec = self.Enc(Anchor)
        SampleVec = self.Enc(Sample)
        x = torch.cat((AnchorVec, SampleVec), 1)
        x = self.linear(x)
        return x
        
./previous_models/ScoreNet3.pth        
class ScoreNet(nn.Module):
    def __init__(self, Encoder_Path):
        super(ScoreNet, self).__init__()
        self.Enc = Encoding_Network()
        self.Enc.load_state_dict(torch.load(Encoder_Path))
        
        self.linear = nn.Sequential(nn.Linear(512, 1024), nn.BatchNorm1d(1024), nn.ReLU(), nn.Linear(1024,512),
                                    nn.BatchNorm1d(512), nn.ReLU(), nn.Linear(512,1) )
        
    def forward(self,Anchor, Sample):
        AnchorVec = self.Enc(Anchor)
        SampleVec = self.Enc(Sample)
        x = torch.cat((AnchorVec, SampleVec), 1)
        x = self.linear(x)
        return x
        
'./previous_models/ScoreNet4.pth'   78% .45      
class ScoreNet(nn.Module):#must be paird w long vector
    def __init__(self, Encoder_Path):
        super(ScoreNet, self).__init__()
        self.Enc = Longer_Encoding_Network()
        self.Enc.load_state_dict(torch.load(Encoder_Path))
        
        self.linear = nn.Sequential(nn.Linear(1568, 3012), nn.BatchNorm1d(3012), nn.ReLU(), nn.Linear(3012,512), nn.BatchNorm1d(512),nn.ReLU(), nn.Linear(512,1) )
        
    def forward(self,Anchor, Sample):
        AnchorVec = self.Enc(Anchor)
        SampleVec = self.Enc(Sample)
        x = torch.cat((AnchorVec, SampleVec), 1)
        x = self.linear(x)
        return x
'''